In [1]:
# switch to the project directory
%cd ..
# working directory should be ../pdi

c:\Users\admin\Desktop\research\pdi


In [2]:
import sys
import os

module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pdi.constants import (
    PARTICLES_DICT,
    TARGET_CODES,
    NUM_WORKERS,
)

In [4]:
config_common = {
    "bs": 512,
    "max_epochs": 1,  # 40
    "dropout": 0.1,
    "gamma": 0.9,
    "patience": 5,
    "patience_threshold": 0.001,
}

In [5]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
device = torch.device("cuda")

In [6]:
from pdi.data.preparation import FeatureSetPreparation, MeanImputation, DeletePreparation, RegressionImputation, EnsemblePreparation
from pdi.models import AttentionModel, NeuralNetEnsemble, NeuralNet
from pdi.data.constants import N_COLUMNS
from pdi.data.types import Split

EXPERIMENTS = {
    "Delete": {
        "data_preparation":
        DeletePreparation(),
        "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "model_class":
        NeuralNet,
        "model_args":
        lambda d_prep: [[
            N_COLUMNS, wandb.config.h0, wandb.config.h1, wandb.config.h2, 1
        ], nn.ReLU, wandb.config.dropout]
    },
    "Mean": {
        "data_preparation":
        MeanImputation(),
        "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "model_class":
        NeuralNet,
        "model_args":
        lambda d_prep: [[
            N_COLUMNS, wandb.config.h0, wandb.config.h1, wandb.config.h2, 1
        ], nn.ReLU, wandb.config.dropout]
    },
    "Regression": {
        "data_preparation":
        RegressionImputation(),
        "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "model_class":
        NeuralNet,
        "model_args":
        lambda d_prep: [[
            N_COLUMNS, wandb.config.h0, wandb.config.h1, wandb.config.h2, 1
        ], nn.ReLU, wandb.config.dropout],
    },
    "Ensemble": {
        "data_preparation":
        EnsemblePreparation(),
        "config": {
            "h0": 64,
            "h1": 32,
            "h2": 16,
            "start_lr": 5e-4,
        },
        "model_class":
        NeuralNetEnsemble,
        "model_args":
        lambda d_prep: [
            d_prep.get_group_ids(),
            [wandb.config.h0, wandb.config.h1, wandb.config.h2, 1],
            nn.ReLU,
            wandb.config.dropout,
        ],
    },
    "Proposed": {
        "data_preparation":
        FeatureSetPreparation(),
        "config": {
            "embed_in": N_COLUMNS + 1,
            "embed_hidden": 128,
            "d_model": 32,
            "ff_hidden": 128,
            "pool_hidden": 64,
            "num_heads": 2,
            "num_blocks": 2,
            "start_lr": 2e-4,
        },
        "model_class":
        AttentionModel,
        "model_args":
        lambda d_prep: [
            wandb.config.embed_in,
            wandb.config.embed_hidden,
            wandb.config.d_model,
            wandb.config.ff_hidden,
            wandb.config.pool_hidden,
            wandb.config.num_heads,
            wandb.config.num_blocks,
            nn.ReLU,
            wandb.config.dropout,
        ],
    },
}


In [7]:
import wandb
import os
from pdi.train import train
from pdi.constants import PARTICLES_DICT


def do_train(experiment_name, data_preparation, config, model_class,
             model_args):
    wandb_config = {**config_common, **config}

    train_loader, val_loader = data_preparation.prepare_dataloaders(
        wandb_config["bs"], NUM_WORKERS, [Split.TRAIN, Split.VAL])

    for target_code in TARGET_CODES:
        save_path = f"models/{experiment_name}/{PARTICLES_DICT[target_code]}.pt"
        with wandb.init(project=experiment_name,
                        config=wandb_config,
                        name=PARTICLES_DICT[target_code]) as run:
            # pos_weight = torch.tensor(data_preparation.pos_weight(target_code)).float().to(device)
            pos_weight = torch.tensor(1.0).to(device)
            wandb.log({"pos_weight": pos_weight.item()})

            model_init_args = model_args(data_preparation)
            model = model_class(*model_init_args).to(device)

            os.makedirs(f"models/{experiment_name}/", exist_ok=True)
            train(model, target_code, device, train_loader, val_loader,
                  pos_weight)

            save_dict = {
                "state_dict": model.state_dict(),
                "model_args": model_init_args,
                "model_thres": model.thres
            }

            torch.save(save_dict, save_path)

In [9]:
do_train("Delete", **EXPERIMENTS["Delete"])

  0%|          | 0/1098 [00:04<?, ?it/s]


In [ ]:
do_train("Mean", **EXPERIMENTS["Mean"])

100%|██████████| 807/807 [00:08<00:00, 94.08it/s] 


Epoch: 0, F1: 0.9634


epoch,▁
loss,█████▇▇▇▇▆▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,4.54919


100%|██████████| 807/807 [00:08<00:00, 94.52it/s] 


Epoch: 0, F1: 0.5783


...

Epoch: 0, F1: 0.6951


epoch,▁
loss,█▆▇▆▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,1.56353


100%|██████████| 807/807 [00:08<00:00, 97.54it/s] 


Epoch: 0, F1: 0.5239


epoch,▁
loss,█▃▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,3.6604


In [ ]:
do_train("Regression", **EXPERIMENTS["Regression"])

100%|██████████| 807/807 [00:08<00:00, 95.41it/s] 


Epoch: 0, F1: 0.9684


epoch,▁
loss,███▇▇▇▆▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,5.12427


100%|██████████| 807/807 [00:08<00:00, 93.57it/s] 


Epoch: 0, F1: 0.7902


...

Epoch: 0, F1: 0.6899


epoch,▁
loss,█▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,1.83896


100%|██████████| 807/807 [00:08<00:00, 93.52it/s] 


Epoch: 0, F1: 0.5423


epoch,▁
loss,█▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,3.68352


In [ ]:
do_train("Ensemble", **EXPERIMENTS["Ensemble"])

100%|██████████| 808/808 [00:15<00:00, 53.49it/s] 


Epoch: 0, F1: 0.9514


epoch,▁
loss,█▇▆▆▆▆▆▆▆▆▆▆▆▅▅▅▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,5.54066


100%|██████████| 808/808 [00:14<00:00, 54.22it/s] 


Epoch: 0, F1: 0.5328


...

Epoch: 0, F1: 0.5200


epoch,▁
loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,1.86656


100%|██████████| 808/808 [00:14<00:00, 54.66it/s] 


Epoch: 0, F1: 0.3909


epoch,▁
loss,█▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,4.16993


In [ ]:
do_train("Proposed", **EXPERIMENTS["Proposed"])

100%|██████████| 807/807 [00:15<00:00, 53.62it/s] 


Epoch: 0, F1: 0.9807


epoch,▁
loss,█▇▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,2.14985


100%|██████████| 807/807 [00:14<00:00, 53.88it/s] 


Epoch: 0, F1: 0.7977


...

Epoch: 0, F1: 0.8847


epoch,▁
loss,█▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,0.69774


100%|██████████| 807/807 [00:13<00:00, 60.53it/s] 


Epoch: 0, F1: 0.7832


epoch,▁
loss,█▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
pos_weight,▁
scheduled_lr,▁
val_f1,▁
val_loss,▁
val_precision,▁
val_recall,▁
val_threshold,▁
epoch,0
loss,1.42385
